<div style="text-align: right"><i>Peter Norvig</i></div>

# Portmantout Words

A [***portmanteau***](https://en.wikipedia.org/wiki/Portmanteau) is a word that squishes together two words, like  *math* + *athlete* = *mathlete*, or *tutankhamenability*, the property of being amenable to seeing the Egyptian exhibit.  Inspired by [**Darius Bacon**](http://wry.me/), I covered this as a programming exercise in my 2012 [Udacity course](https://www.udacity.com/course/design-of-computer-programs--cs212). Recently I was re-inspired by [**Tom Murphy VII**](http://tom7.org/), who added a new twist:  [***portmantout words***](http://www.cs.cmu.edu/~tom7/portmantout/) (*tout* from the French for *all*), which are defined as:

> A **portmantout** of a set of words $W$ is a string $S$ such that:
* Every word in $W$ is a **substring** of $S$.
* The words **overlap**: each word (except the first) must start at an index that is between the beginning and end of another word.
* **Nothing else** is in $S$: every letter in $S$ comes from the overlapping words. (But a word may be repeated any number of times.)

Although not part of the definition, the goal is to get as short an $S$ as possible, and to do it for a $W$ of 100,000 words or so. Developing a program to do that is the goal of this notebook.  My program (also available as [portman.py](portman.py)) helped me discover:

- **preferendumdums**: a political commentary portmantout of {prefer, referendum, dumdums}
- **fortyphonshore**: a dire weather report portmantout of {forty, typhons, onshore}; 
- **allegestionstage**: a brutal  theatre critic portmantout of {alleges, egestions, onstage}. 
- **skymanipulablearsplittingler**: a nerve-damaging aviator portmantout of {skyman, manipulable, blears, earsplitting, tinglers}
- **edinburgherselflesslylyricize**: a Scottish music review portmantout of {edinburgh, burghers, herself, selflessly, slyly, lyricize}



# Program Design

I originally thought I would define a major function, `S = portman(W)`, to generate the portmantout string, and a minor function, `is_portman(W, S)`, to verify the result. But I found the verification process was difficult. For example, given  `S = '...helloworld...'` I would reject that as non-overlapping if I parsed it as `'hello'` + `'world'`, but I would accept it if parsed as `'hell'` + `'low'` + `'world'`. It was hard for `is_portman` to decide which parse was intended, which is a shame because `portman` *knew* which was intended, but  discarded the information. 

Therefore, I decided to change the interface: I'll have one function that takes $W$ as input and returns what I call a **portmantout proof**, $P$. I can gain insight by examining $P$, and I can pass $P$ to a second function that can easily generate the string $S$ while verifying the proof.  I decided on the following calling and [naming](https://en.wikipedia.org/wiki/Natalie_Portman) conventions:

    P = natalie(W)     # Generate a portmantout proof P from a set of words W
    S = portman(P, W)  # Verify that the proof is valid and compute the string S

or in other words:

    S = portman(natalie(W), W) # Generate a portmantout of W

The proof $P$ is in the form of an ordered list, `[(overlap, word),...]` where each `word` is a member of $W$ and each `overlap` is an integer saying how many letters in the word overlap with the previous word (this should be 0 for the first word and positive for subsequent words). For example:

In [1]:
Words = set 
Proof = list

W1: Words = {'anarchy', 'eskimo', 'grammarian', 'kimono', 'monogram'}
S1: str   =      'eskimonogrammarianarchy'
P1: Proof = [(0, 'eskimo'),
             (4,   'kimono'),
             (4,     'monogram'),
             (4,         'grammarian'),
             (2 ,                'anarchy')]

# portman

The function `portman(P, W)` takes a proof $P$ and a set of words $W$ and generates the portmantout string $S$ while verifying that the proof is correct (or raising an `AssertionError` if it is not). Assertions are appropriate because I'm thinking of this as one part of my program verifying the internal logic of another part. If I was running a service to verify other people's proofs, I would not use `assert` statements.

In [2]:
from collections import defaultdict, Counter

In [3]:
def portman(P: Proof, W: Words) -> str:
    """Compute the portmantout string S from the proof P; verify that it covers W."""
    S = []
    prev_word = ''
    for (overlap, word) in P:
        assert word in W, f'nothing else is allowed in S: {word}'
        left, right = word[:overlap], word[overlap:] # Split word into two parts
        assert overlap >= 0 and left == prev_word[-overlap:], f'the words must overlap: {prev_word, word}'
        S.append(right)
        prev_word = word
    S = ''.join(S)
    assert all(w in S for w in W), 'each word in W must be a substring of S'
    return S

In [4]:
portman(P1, W1)

'eskimonogrammarianarchy'

In [5]:
portman([(0, 'hell'), (1, 'low'), (1, 'world')], {'hell', 'low', 'world'})

'helloworld'

# Subwords

I want to introduce the concept of **subwords**. The following set of words `W2` has 17 more words than `W1`:

In [6]:
W2 = {'anarchy', 'eskimo', 'grammarian', 'kimono', 'monogram',  
      'a', 'am', 'an', 'arc', 'arch', 'aria', 'gram', 'grammar', 
      'i', 'mar', 'mono', 'narc', 'no', 'on', 'ram', 'ski', 'skim'}

But our old `P1` still works as a portmantout proof of `W2`, yielding the same string:

In [7]:
portman(P1, W2)

'eskimonogrammarianarchy'

This works because the 17 new words in `W2`  are all **subwords** of the first five words. If a superword like `'monogram'` is included in the proof $P$ and thus in the string $S$, then subwords like `'on'`, `'no'`, and `'gram'` are automatically included, without having to explicitly list them in $P$. 

We can compute the subwords (and from that, nonsubwords) of a set of words as follows:

In [8]:
def subwords(W: Words) -> Words:
    """All the words in W that are subparts of some other word."""
    wordparts = (subparts(w) & W for w in W)
    return set().union(*wordparts)

def subparts(word) -> set:
    """All non-empty proper substrings of this word"""
    return {word[i:j] 
            for i in range(len(word)) 
            for j in range(i + 1, len(word) + (i > 0))}

(*Python trivia:* the `(i > 0)` in `subparts` means that for a four-letter word like `'skim'`, we include subparts `word[i:4]` except when `i == 0`, thus including `'kim'`, `'im'`, and `'m'`, but not `'skim'`. In Python it is considered good style to have a Boolean expression like `(i > 0)` automatically converted to an integer `0` or `1`.)

(*English trivia:* I use the clumsy term "nonsubwords" rather than "superwords", because there are a couple dozen words, like "cozy" and "pugs" and "july," that are not subwords of any other words but are also not superwords: they have no subwords.)

In [9]:
subparts('skim') # The subparts (non-empty proper substrings) of a word

{'i', 'im', 'k', 'ki', 'kim', 'm', 's', 'sk', 'ski'}

In [10]:
W2 - subwords(W2) # These nonsubwords must be in the proof

{'anarchy', 'eskimo', 'grammarian', 'kimono', 'monogram'}

In [11]:
subwords(W2) # These subwords don't need to appear in a proof

{'a',
 'am',
 'an',
 'arc',
 'arch',
 'aria',
 'gram',
 'grammar',
 'i',
 'mar',
 'mono',
 'narc',
 'no',
 'on',
 'ram',
 'ski',
 'skim'}

Now that we have the notion of subwords, we can modify `portman` to be a bit more efficient and concise: 

In [12]:
def portman(P: Proof, W: Words) -> str:
    """Compute the portmantout string S from the proof P; verify that it covers W."""
    assert (W - subwords(W)) <= set(w for _, w in P) <= W, "all the words in W and nothing else"
    S = []
    prev_word = ''
    for (overlap, word) in P:
        left, right = word[:overlap], word[overlap:] # Split word into two parts
        assert overlap >= 0 and left == prev_word[-overlap:], f'the words must overlap: {prev_word, word}'
        S.append(right)
        prev_word = word
    return ''.join(S)

In [13]:
portman(P1, W2)

'eskimonogrammarianarchy'

(*Python trivia:* if `X, Y` and `Z` are sets, `X <= Y <= Z` means "is `X` a subset of `Y` and `Y` a subset of `Z`?" We use the notation here to say that the set of words in $P$ must contain all the nonsubwords and can only contain words from $W$.)

# Set of 108,709 Words

I will fetch the set of words that Tom Murphy used, and explore it a bit:

In [14]:
! [ -e wordlist.asc ] || curl -O https://norvig.com/ngrams/wordlist.asc

In [15]:
W = set(open('wordlist.asc').read().split())

In [16]:
N = len(W)
sub = len(subwords(W))

f'W has {N:,d} words ({sub:,d} subwords and {N-sub:,d} nonsubwords)'

'W has 108,709 words (44,320 subwords and 64,389 nonsubwords)'

In [17]:
'eskimo' in W

True

In [18]:
'waldo' in W # Where's Waldo? Not in W

False

In [19]:
max(W, key=len)

'antidisestablishmentarianism'

In [20]:
Counter(sorted(map(len, W))) # word lengths

Counter({1: 2,
         2: 25,
         3: 500,
         4: 2920,
         5: 6826,
         6: 11454,
         7: 16852,
         8: 19445,
         9: 16684,
         10: 11876,
         11: 8372,
         12: 5811,
         13: 3676,
         14: 2101,
         15: 1159,
         16: 583,
         17: 229,
         18: 107,
         19: 39,
         20: 29,
         21: 11,
         22: 4,
         23: 2,
         25: 1,
         28: 1})

In [21]:
Counter(w[0] for w in W).most_common() # first letters

[('s', 12075),
 ('c', 10287),
 ('p', 8414),
 ('r', 6772),
 ('d', 6673),
 ('a', 6333),
 ('b', 6205),
 ('m', 5757),
 ('t', 5490),
 ('f', 4691),
 ('e', 4460),
 ('i', 4354),
 ('h', 3884),
 ('g', 3576),
 ('l', 3331),
 ('u', 3302),
 ('o', 2935),
 ('w', 2697),
 ('n', 2453),
 ('v', 1799),
 ('j', 1039),
 ('k', 960),
 ('q', 559),
 ('y', 338),
 ('z', 260),
 ('x', 65)]

In [22]:
Counter(w[-1] for w in W).most_common() # last letters

[('s', 35647),
 ('d', 11139),
 ('e', 10701),
 ('y', 10071),
 ('g', 9125),
 ('r', 7643),
 ('t', 5990),
 ('n', 5261),
 ('l', 3314),
 ('c', 1819),
 ('m', 1600),
 ('a', 1398),
 ('h', 1268),
 ('k', 920),
 ('p', 699),
 ('o', 665),
 ('i', 343),
 ('w', 306),
 ('x', 243),
 ('f', 240),
 ('b', 158),
 ('u', 88),
 ('z', 51),
 ('v', 15),
 ('j', 3),
 ('q', 2)]

In [23]:
Counter(L for w in W for L in w).most_common() # all letters

[('e', 108278),
 ('s', 83059),
 ('i', 81747),
 ('a', 70762),
 ('r', 68864),
 ('n', 64756),
 ('t', 61999),
 ('o', 57205),
 ('l', 50145),
 ('c', 37734),
 ('d', 34192),
 ('u', 31126),
 ('g', 26858),
 ('p', 26294),
 ('m', 25538),
 ('h', 20654),
 ('b', 18169),
 ('y', 15752),
 ('f', 12757),
 ('v', 9670),
 ('k', 8109),
 ('w', 7971),
 ('z', 4083),
 ('x', 2666),
 ('j', 1746),
 ('q', 1689)]

# natalie

The function `natalie` generates a portmantout proof for a set of words. The rough outline is:

    def natalie(W: Words) -> Proof:
        precompute some data structures to make things more efficient
        P = a proof, initially with just the first word, that we will build up
        while there are nonsubwords that have not been used:
            for (overlap, word) in (unused or bridging words that overlap):
                append (overlap, word) to P and update data structures
        return P
    
There are two choices of how to pick a word to add to `P`:
- The function `unused_word` finds a word that has not been used yet that has a maximal overlap with the previous word. If there is such a word, we will always use it, and never consider reverting that choice. That's called a **greedy** approach, and it typically leads to solutions that are not optimal (the resulting $S$ is not the shortest possible) but are computationally feasible. It seems like finding a shortest $S$ is an NP-hard problem, and with 100,000 words to cover, it is unlikely that I can find an optimal solution in a reasonable amount of time. So I'm happy with the greedy, suboptimal approach.
- The function `bridging_words` is called only when there is no way to add an unused word to the previous word. `bridging_words` returns a one- or two-word sequence (called a **bridge**) that will bring us to a place where we can again consume an unused word on the following iteration.  

`natalie` keeps track of the following data structures (which we will explain in more detail below):
- `unused: Words`: a set of the unused nonsubwords in $W$. When a word is added to $P$, it is removed from `unused`.
- `P: Proof`: e.g. `[(0, 'eskimo'), (4, 'kimono'),...]`, the proof that we are building up.
- `startswith: dict`: e.g. `startswith['kimo'] = ['kimono',...]` is a list of words that start with `'kimo'`. 
- `firsts: Counter`: e.g. `firsts['a'] == 3528` is the number of unused words that start with the letter `a`.
- `bridges: dict`: e.g. `bridges['a' + 'q'] == ('airaq', [(1, 'air'), (2, 'iraq')])`, a description of a way to bridge from a word that ends in `'a'` to one that begins with `'q'`.

In [24]:
def natalie(W: Words, start=None) -> Proof:
    """Return a portmantout string for W, and a Proof for it."""
    prev_word  = start or next(iter(W))
    unused     = W - (subwords(W) | {prev_word})
    P: Proof   = [(0, prev_word)] # The emerging Proof   
    startswith = compute_startswith(unused) # startswith['th'] = words that start with 'th'
    firsts     = Counter(word[0] for word in unused) # Count of first letters of words
    bridges    = compute_bridges(W) # Words that bridge from 'a' to 'b'
    while unused:
        for (overlap, word) in (unused_word(prev_word, startswith, unused) or
                                bridging_words(prev_word, firsts, bridges)):
            if word not in W:
                return [] # Fail
            P.append((overlap, word))
            if word in unused:
                unused.remove(word)
                firsts -= Counter(word[0])
            prev_word = word
    return P

(*Python trivia:* I say `firsts -= Counter(word[0])` rather than `first[word[0]] -= 1` because when the count for a letter reaches zero, the former deletes the letter from the Counter, whereas the latter just sets it to zero.)

How do we know that `unused` will eventually be empty, so that the `while unused` loop can terminate? On each iteration we either use `unused_word`, which reduces the size of `unused`, or we use `bridging_words`, which doesn't. But after `bridging_words` adds the bridging word(s), we are guaranteed to be able to use an `unused` word on the next iteration (at least for the word set $W$; for smaller word sets `bridging_words` might return a non-word, which causes `natalie` to return the empty proof, indicating failure. 

The most important parts of `natalie` are the two functions `unused_word` and `bridging_words`, which decide what words will be added next to the emerging proof. Both return a list of the form `[(overlap, word),...]` that define a word or sequence of words to add to the proof, and the number of letters that each word overlaps the previous word. We will discuss the two functions in turn.

# unused_word and compute_startswith

To select an `unused_word`, consider the suffixes of the previous word, longest suffix first, and if that suffix is a prefix of an unused word, then take it. `unused_word` returns either a list of length one, `[(overlap, word)]`, or the empty list, `[]` if no overlapping word can be found. 

In [25]:
def unused_word(prev_word: str, startswith: dict, unused: Words) -> [(int, str),...]:
    """Return a [(overlap, word)] pair to follow `prev_word`, or []."""
    return next(([(len(suf), word)]
                 for suf in suffixes(prev_word) if suf in startswith
                 for word in startswith[suf] if word in unused), [])

def suffixes(word) -> list:
    """All non-empty proper suffixes of word, longest first."""
    return [word[i:] for i in range(1, len(word))]

def prefixes(word) -> list:
    """All non-empty proper prefixes of word, shortest first."""
    return [word[:i] for i in range(1, len(word))]

def multimap(pairs) -> dict:
    """Given (key, val) pairs, make a dict of {key: [val,...]}."""
    result = defaultdict(list)
    for key, val in pairs:
        result[key].append(val)
    return result

def compute_startswith(W) -> dict: 
    """A mapping of a prefix to a list of the words that start with it."""
    return multimap((pre, w) for w in W for pre in prefixes(w))

 So when the previous word is `'eskimo'`, a call to `unused_word` finds a word with a four-letter overlap; no other word overlaps more:

In [26]:
startswith = compute_startswith(W)

unused_word('eskimo', startswith, W)

[(4, 'kimono')]

Some examples of usage:

In [27]:
{w: unused_word(w, startswith, W) 
 for w in '''eskimo kimono shoehorns elephant phantasies dachshund vicars 
             flimsiest siesta dilettantism antismog seascape snark referendum'''.split()}

{'eskimo': [(4, 'kimono')],
 'kimono': [(4, 'monomers')],
 'shoehorns': [(5, 'hornswoggling')],
 'elephant': [(5, 'phantasm')],
 'phantasies': [(4, 'siesta')],
 'dachshund': [(4, 'hundredfold')],
 'vicars': [(4, 'carsick')],
 'flimsiest': [(5, 'siesta')],
 'siesta': [(4, 'establisher')],
 'dilettantism': [(6, 'antismog')],
 'antismog': [(4, 'smoggier')],
 'seascape': [(5, 'scapes')],
 'snark': [(4, 'narks')],
 'referendum': [(3, 'dumpcart')]}

In [28]:
startswith['kimo']

['kimono', 'kimonos', 'kimonoed']

In [29]:
suffixes('kimono')

['imono', 'mono', 'ono', 'no', 'o']

In [30]:
prefixes('kimono')

['k', 'ki', 'kim', 'kimo', 'kimon']

In [31]:
{pre: startswith[pre] # Rare two-letter prefixes
 for pre in startswith if len(pre) == 2 and len(startswith[pre]) <= 2}

{'uf': ['ufos', 'ufo'],
 'gj': ['gjetost', 'gjetosts'],
 'oj': ['ojibwas', 'ojibwa'],
 'ry': ['rye'],
 'pf': ['pfennig', 'pfennigs'],
 'yc': ['ycleped', 'yclept'],
 'zl': ['zloty', 'zlotys'],
 'mc': ['mcdonald'],
 'ez': ['ezekiel'],
 'fj': ['fjord', 'fjords'],
 'tc': ['tchaikovsky'],
 'xm': ['xmases', 'xmas'],
 'ie': ['ieee'],
 'dn': ['dnieper'],
 'ud': ['udder', 'udders'],
 'sf': ['sforzato', 'sforzatos'],
 'aj': ['ajar'],
 'ym': ['ymca'],
 'vy': ['vyingly', 'vying'],
 'qo': ['qophs', 'qoph'],
 'hd': ['hdqrs'],
 'zw': ['zwieback', 'zwiebacks'],
 'jn': ['jnana', 'jnanas'],
 'dv': ['dvorak'],
 'bw': ['bwana', 'bwanas'],
 'fb': ['fbi'],
 'ct': ['ctrl']}

# bridging_words and compute_bridges

Suppose we reach a situation where the previous word was `'one'`, and the only remaining unused words are `'two'`, `'three'`, and `'six'`. Since there is no possible overlap, `unused_word` will return the empty list:

In [32]:
unused_word('one', startswith, {'two', 'three', 'six'})

[]

However, we can still hope to find a previously-used a word that will **bridge** from the `'e'` at the end of `'one'` to the `'t'` or `'s'` at the start of one of the unused words.  The function `compute_bridges` precomputes a table that we call `bridges`, and the function `bridging_words` finds the approriate bridging word(s) in that table.

The `bridging_words` function is simple: in this example we fetch `bridges['e' + 't']` and `bridges['e' + 's']`, decide which gives us the shortest bridge, and return the list `[(overlap, word),...]` that makes up that bridge. For `unused_word` the list was always of length zero or one; for `bridging_words` the list is always of length one or two.

To compute the bridges is a bit of work, but we only have to do it once. We want a table of `{A + B: (bridge, [(overlap, word),...])}` where `A` and `B` are the letters we want to bridge between, and `bridge` is either a single word or a portmanteau of two words. 

We start by selecting from $W$ all the short words, as well as words that end in an unusual letter. (For our 108,709  word set $W$, we selected the 10,273 words with length up to 5, plus 159 words that end in any of 'qujvz', the five rarest letters. For other word sets, you may have to tune these parameters.) We consider each of these individual words as a possible bridge between its first and last letters, keeping only the shortest. The following means that `'eat'` is a shortest possible word that starts with `'e'` and ends in `'t'`.

     bridges['e' + 't'] == ('eat', [(1, 'eat')])
     
Sometimes we can't bridge with one word: no word starts with `'a'` and ends with `'q'`. Thus, we also consider two-word bridges:

     bridges['a' + 'q'] == ('airaq', [(1, 'air'), (2, 'iraq')])
     
which means that the shortest possible portmanteau that starts with `'a'` and ends in `'q'` is `'airaq'`, and the first word in that portmanteau is `'air'` and the second `'iraq'`.

Here's the code:

In [33]:
def bridging_words(prev: str, firsts: Counter, bridges: dict) -> [(int, str),...]:
    """Find a previously-used word that will bridge to one of the letters in firsts."""
    A = prev[-1] # Ending letter of previous word
    (bridge, pairs) = min((bridges[A + B] for B in firsts), key=bridgelen)
    return pairs

def bridgelen(bridge) -> int: return len(bridge[0])
    
def compute_bridges(W: Words, maxlen=5, endings=tuple('qujvz')) -> dict:
    """A table of {A + B: (bridge, word1)} pairs that bridge letter A to letter B, 
    e.g. {'e'+'t': ('eat', 'eat'), 'a'+'q': ('airaq', 'air')}."""
    long = '?' * 29 # A default long "word"
    bridges = {A + B: (long, [(1, long)]) for A in alphabet for B in alphabet}
    shortwords = [w for w in W if len(w) <= maxlen or w.endswith(endings)]
    startswith = compute_startswith(shortwords)
    
    def consider(bridge, *pairs):
        """Use this bridge if it is shorter than the previous bridges[AB]."""
        AB = bridge[0] + bridge[-1]
        bridges[AB] = min(bridges[AB], (bridge, [*pairs]), key=bridgelen)
        
    for w1 in shortwords:
        consider(w1, (1, w1)) # One-word bridges
        for suf in suffixes(w1): 
            for w2 in startswith[suf]:
                bridge = w1 + w2[len(suf):]
                consider(bridge, (1, w1), (len(suf), w2)) # Two-word bridges
    return bridges

alphabet = 'abcdefghijklmnopqrstuvwxyz'

Here is how `bridging_words` works in our example situation. First the necessary data structures `firsts` and `bridges`:

In [34]:
firsts = Counter(t=2, s=2) # Counts of the first letters of unused words in this situation

%time bridges = compute_bridges(W)

CPU times: user 7.17 s, sys: 75 ms, total: 7.24 s
Wall time: 7.36 s


In [35]:
bridging_words('one', firsts, bridges)

[(1, 'eat')]

That says that we should add the word `'eat'`, which overlaps one letter with `one`. After adding `'eat'`, we'll be set up to add one of the unused words on the next step. Here's some of the calculations that got us to `'eat'`:

In [36]:
bridges['e' + 't']

('eat', [(1, 'eat')])

In [37]:
bridges['e' + 's']

('ells', [(1, 'ells')])

In [38]:
A = 'e'
min((bridges[A + B] for B in firsts), key=bridgelen)

('eat', [(1, 'eat')])

Some more examples of bridges:

In [39]:
bridges['a' + 'q']

('airaq', [(1, 'air'), (2, 'iraq')])

In [40]:
bridges['y' + 'u']

('you', [(1, 'you')])

In [41]:
max(bridges.values(), key=bridgelen)

('xericolloq', [(1, 'xeric'), (1, 'colloq')])

In [42]:
Counter(map(bridgelen, bridges.values())).most_common() # How long are bridges?

[(3, 261),
 (4, 256),
 (5, 69),
 (6, 34),
 (2, 25),
 (7, 21),
 (8, 7),
 (1, 2),
 (10, 1)]

In [43]:
{AB: bridges[AB] for AB in bridges if bridgelen(bridges[AB]) >= 8} # Longest bridges

{'bq': ('bocciraq', [(1, 'bocci'), (1, 'iraq')]),
 'gq': ('geniiraq', [(1, 'genii'), (1, 'iraq')]),
 'jq': ('jinniraq', [(1, 'jinni'), (1, 'iraq')]),
 'oq': ('obeliraq', [(1, 'obeli'), (1, 'iraq')]),
 'qq': ('quasiraq', [(1, 'quasi'), (1, 'iraq')]),
 'vj': ('vetchajj', [(1, 'vetch'), (1, 'hajj')]),
 'xj': ('xericonj', [(1, 'xeric'), (1, 'conj')]),
 'xq': ('xericolloq', [(1, 'xeric'), (1, 'colloq')])}

*Note 1:* My `compute_bridges` only does one-letter-to-one-letter bridges. It just seemed simpler to only fill in a 26×26 `bridges` table, and only maintain 26 entries in `firsts`.  But that means sometimes we get a long bridge when we could have found a shorter one with more work. Here's an example where the previous word is `cogito` and there's only one word left, `'question'`, which starts with `'q'`. We end up with the bridge `'oculiraq'`, adding 6 letters between the `'o'` and `'q'`:

In [44]:
bridging_words('cogito', {'q': 1}, bridges)

[(1, 'obeli'), (1, 'iraq')]

But if we had arranged for multi-letter-to-multi-letter bridges, we could have noticed that the bridge `'toque'`  adds *zero* letters between the `'to'` at the end of `'cogito'` and the `'que'` at the start of `'question'`. How could I find `'toque'` in this situation? One approach would be to precompute a `bridges` table with up to two letters on the left and three on the right. That would mean about $26^5 = 12$ million table entries (although most of them will be empty). Another approach would be to call `unused_word`, but give it a selection of words that have been used before, and then check if it ends up in a place that allows us to make progress on the next move. On the one hand, this seems like a promising idea to explore. On the other hand, I did a quick measurement, and the average number of letters added per bridge under the current algorithm is just about 1, so there's not that much room to improve. Maybe this approach could reduce the number of letters in $S$ by 1%.

*Note 2:* I should say that I started watching Tom Murphy's highly-entertaining [video](https://www.youtube.com/watch?time_continue=1&v=QVn2PZGZxaI), but then I paused it because I wanted the fun of solving the problem mostly on my own. After I finished the first version of my program, I returned to the video and [paper](http://tom7.org/portmantout/murphy2015portmantout.pdf) and I noticed that 
Murphy had an approach to bridges that was very similar to mine, but in one way clearly superior:  In my original program I only considered two-word bridges when there was no one-word bridge, on the theory that one word is shorter than two.  My program looked something like:

    (overlap, word) = unused_word(...) or one_word_bridge(...) or two_word_bridge(...)
    
But Murphy showed that my theory was wrong: I had `bridges['w' + 'c'] = 'workaholic'`, but he had  `'warc'`, a portmanteau of  `'war'` and `'arc'`, which saves six letters over my single word. After seeing this, I shamelessly copied his approach, and now I too get a four-letter bridge for `'w' + 'c'` (sometimes  `'warc'` and sometimes `'wet' + 'etc' = 'wetc'`), and my portmantout strings are about 0.5% shorter.

In [45]:
bridges['w' + 'c']

('wetc', [(1, 'wet'), (2, 'etc')])

(*English trivia:*  my program builds a single path of words, and when the path gets stuck and I need something to allow me to continue, it makes sense to call that thing a **bridge**.  Murphy's program starts by building a large pool of small portmanteaux that he calls **particles**, and when he can build no more particles, his next step is to put two particles together, so he calls it a **join**. The different metaphors for what our programs are doing lead to different terminology for the same idea.)

# A Solution

We're finally ready to solve problems! First the tiny `W2` word set, for which we must specify the starting word, or it will fail:

In [46]:
natalie(W2, start='eskimo')

[(0, 'eskimo'),
 (4, 'kimono'),
 (4, 'monogram'),
 (4, 'grammarian'),
 (2, 'anarchy')]

In [47]:
portman(natalie(W2, start='eskimo'), W2)

'eskimonogrammarianarchy'

Now the big `W` word set, which works fine when it selects a starting word on its own, but, following Tom Murphy and doing him one letter better, I will supply the starting word `portmanteaux`:

In [48]:
%%time
P = natalie(W, start='portmanteaux')
S = portman(P, W)
len(S)

CPU times: user 24.4 s, sys: 89.4 ms, total: 24.5 s
Wall time: 24.6 s


570002

A little over half a million letters long. Here is the start of the string $S$:

In [49]:
S[:500]

'portmanteauxiliariespritselfwardressestetsonshipsideswipingrassessorshipkeepergnestlikeneditorializationshorelinesmantillascarsonickeledematouslessoneditorializestfullynchessmanganousefulnessesquicentenniallylstonedifiersatzestygiantismshirtwaistlinesmenadsorptionosphericitywardenshipbuilderstwhiledgaroteddyingsinewsstandstilliestablisheriffdomicilingamsterdamnitrosemariestablishmentsaritzaspersersafaristocraciestimatorsionallyratelysianthologizedswampinessentiallyonnaisecularizershankingwoodwax'

It is easier to grasp by looking at the first 50 items in  $P$:

In [50]:
P[:50]

[(0, 'portmanteaux'),
 (3, 'auxiliaries'),
 (2, 'esprits'),
 (3, 'itself'),
 (4, 'selfward'),
 (4, 'wardresses'),
 (3, 'sestets'),
 (5, 'stetsons'),
 (4, 'sonships'),
 (5, 'shipside'),
 (4, 'sideswiping'),
 (4, 'pingrasses'),
 (5, 'assessorship'),
 (4, 'shipkeeper'),
 (4, 'epergnes'),
 (3, 'nestlike'),
 (4, 'likened'),
 (2, 'editorializations'),
 (3, 'onshore'),
 (5, 'shorelines'),
 (5, 'linesman'),
 (3, 'mantillas'),
 (3, 'lascars'),
 (4, 'carson'),
 (5, 'arsonic'),
 (3, 'nickeled'),
 (2, 'edematous'),
 (4, 'tousles'),
 (3, 'lessoned'),
 (2, 'editorializes'),
 (3, 'zestfully'),
 (2, 'lynches'),
 (4, 'chessman'),
 (3, 'manganous'),
 (2, 'usefulness'),
 (7, 'fulnesses'),
 (3, 'sesquicentennially'),
 (4, 'allyls'),
 (1, 'stoned'),
 (2, 'edifiers'),
 (3, 'ersatzes'),
 (3, 'zesty'),
 (3, 'stygian'),
 (4, 'giantisms'),
 (1, 'shirtwaist'),
 (5, 'waistlines'),
 (5, 'linesmen'),
 (3, 'menads'),
 (3, 'adsorption'),
 (3, 'ionospheric')]

I'll write the whole string to the file [natalie.txt](natalie.txt):

In [51]:
open('natalie.txt', 'w').write(S)

570002

We can create a report on how we did:

In [52]:
def report(S, W, P):
    sub    = subwords(W)
    nonsub = W - sub
    bridge  = len(P) - len(nonsub)
    def Len(words): return sum(map(len, words))
    print(f'W has {len(W):,d} words ({len(nonsub):,d} nonsubwords; {len(sub):,d} subwords).')
    print(f'P has {len(P):,d} words ({len(nonsub):,d} nonsubwords; {bridge:,d} bridge words).')
    print(f'S has {len(S):,d} letters; W has {Len(W):,d}; nonsubs have {Len(nonsub):,d}.')
    print(f'The average overlap in P is {(Len(w for _,w in P)-len(S))/(len(P)-1):.2f} letters.')
    print(f'The compression ratio (W/S) is {Len(W)/len(S):.2f}.')

In [53]:
report(S, W, P)

W has 108,709 words (64,389 nonsubwords; 44,320 subwords).
P has 105,009 words (64,389 nonsubwords; 40,620 bridge words).
S has 570,002 letters; W has 931,823; nonsubs have 595,805.
The average overlap in P is 1.34 letters.
The compression ratio (W/S) is 1.63.


In [54]:
report(S1, W2, P1)

W has 22 words (5 nonsubwords; 17 subwords).
P has 5 words (5 nonsubwords; 0 bridge words).
S has 23 letters; W has 90; nonsubs have 37.
The average overlap in P is 3.50 letters.
The compression ratio (W/S) is 3.91.


# Next Steps

Each time I restart this notebook, I get a slightly different result. (*Python trivia*: every time you start a new `python` instance, you get a slightly different [`hash`](https://docs.python.org/3.4/reference/datamodel.html#object.__hash__) function, which means that an iteration over a set may yield elements in a different order. This is to prevent a kind of denial of service attack on web servers, but for my program it means that the iteration over `unused` words is in a different order, so the results are different.) I had originally planned to use the **random restart** approach: add some randomness to the word selections and take the best of multiple trials. However, all the trials I have seen so far with my current program fall within a narrow range of 570,000 to 578,000 letters, so I think it is not worth the effort for what would probably be a 1% improvement at best. 

To compare my [program](portman.py) to [Murphy's](https://sourceforge.net/p/tom7misc/svn/HEAD/tree/trunk/portmantout/): I used a greedy approach that incrementally builds up a single long portmanteau, extending it via a bridge when necessary. Murphy first built a pool of smaller portmanteaux, then joined them all together. (I'm reminded of the [Traveling Salesperson Problem](TSP.ipynb) where one algorithm is to form a single path, always progressing to the nearest neighbor, and another algorithm is to maintain a pool of shorter segments and repeatedly join together the two closest segments.) The two approaches are different, but it is not clear whether one is better than the other. 

In terms of implementation, mine is in Python and is concise (112 lines); Murphy's is in C++ and is verbose (1867 lines). Murphy's code does a lot of extra work that mine doesn't: generating diagrams, and running multiple threads in parallel to implement the random restart idea. It appears Murphy didn't quite have the complete concept of **subwords**: he did mention that when he adds `'bulleting'`, he crosses `'bullet'` and `'bulletin'` off the list, but somehow  [his string](http://tom7.org/portmantout/murphy2015portmantout.pdf) contains both `'spectacular'` and `'spectaculars'` in two different places. My guess is that when he adds `'spectaculars'` he crosses off `'spectacular'`, but if he happens to add `'spectacular'` first, he will later add `'spectaculars'`. Support for this view is that his output in `bench.txt` says "I skipped 24319 words that were already substrs", but I computed that there are 44,320 such words; he found about half of them. I think those missing 20,001 words are the main reason why  my shortest string ([570,002 letters](natalie570002.txt)) is about 40,000 letters shorter than his (611,820 letters).

I'll stop here, but you should feel free to do some experimentation of your own. Some ideas:

- With the set of 108,709 words it is always possible to bridge from any letter to any other letter in at most two steps. But for smaller word lists, that might not be the case. You could consider three-word bridges, and consider  what to do when there is no bridging sequence at all (perhaps back up and remove a previously-placed word; perhaps use a beam search to keep several alternatives open at once; perhaps allow the addition of words to the start as well as the end of `P`).
- Use linguistic resources (such as [pretrained word embeddings](https://nlp.stanford.edu/projects/glove/)) to teach your program what words are related to each other. Encourage the program to place  related words next to each other.
- Use linguistic resources (such as [NLTK](https://github.com/nltk/)) to teach your program where syllable breaks are in words. Encourage the program to make overlaps match syllables. (That's why "preferendumdums" sounds better than "fortyphonshore".)
- Here are some ideas to minimize the length of $S$. Can you implement these or think of your own?
  - Get better overlap of bridging words, perhaps by implementing the multi-letter-to-multi-letter approach.
  - Use fewer bridging words by planning ahead to avoid the need for them. Perhaps `compute_startswith` could sort the words in each key's bucket so that the "difficult" words (say, the ones that end in unusual letters) are encountered earlier in the program's execution, when there are more available words for them to connect to.
  - You can't alter the number of nonsubwords, but you might be able to get better overlap between them. Perhaps make more informed choices of which ones to use when. For example, if there is an affix that is the prefix of only one word, and the suffix of only one other word, then probably those two words should overlap.
  - Alter either Murphy's code or mine to combine his idea of joining particles with my idea of completely handling subwords.
  - Use a completely different strategy. What can you come up with?